In [ ]:
# install langchain

In [1]:
from langchain import OpenAI
import os

llm = OpenAI(
    openai_api_key = os.environ["OPENAI_API_KEY"],
    temperature=0,
    model_name="text-davinci-003"
)

KeyError: 'OPENAI_API_KEY'

In [ ]:
from langchain.chains import LLMMathChain
from langchain.agents import Tool

llm_math = LLMMathChain(llm=llm)

# initialize the math tool
math_tool = Tool(
    name='Calculator',
    func=llm_math.run,
    description='Useful for when you need to answer questions about math.'
)
# when giving tools to LLM, we must pass as list of tools
tools = [math_tool]

In [ ]:
tools[0].name, tools[0].description

In [ ]:
from langchain.agents import load_tools

tools = load_tools(
    ['llm-math'],
    llm=llm
)
In[6]:


In [ ]:
tools[0].name, tools[0].description

In [ ]:
from langchain.agents import initialize_agent

zero_shot_agent = initialize_agent(
    agent="zero-shot-react-description",
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3
)

In [ ]:
zero_shot_agent("what is (4.5*2.1)^2.2?")

Check the answer

In [ ]:

(4.5*2.1)**2.2

In [ ]:
zero_shot_agent("if Mary has four apples and Giorgio brings two and a half apple "
                "boxes (apple box contains eight apples), how many apples do we "
                "have?")